# DataBase manipulation
****

This is a notebook in which I have been getting familiar with the the different commands to manipulate a datababase. 
<a id="top"></a>

<b>Table of contents</b><br>

1. [Using basic queries](#basicqueries)

2. [Dealing with dates and times](#dates)

3. [Object-Relational Mapping](#dates) 

2.4 [Constraints](#constraints) 

    


<a id="basicqueries"></a>

### 1.  Basic queries command List 

        1.1. AND|OR          [x]
        1.2. ALTER TABLE
        1.3. AS (alias)
        1.4. BETWEEN
        1.5. CREATE DATABASE [x]
        1.6. CREATE TABLE    [x]
        1.7. CREATE INDEX
        1.8. CREATE VIEW
        1.9. DELETE           [x]
        1.10. GRANT
        1.11. REVOKE
        1.12. COMMIT
        1.13. ROLLBACK
        1.14. SAVEPOINT
        1.15. DROP DATABASE
        1.16. DROP INDEX
        1.17. DROP TABLE       [x]
        1.18. EXISTS
        1.19. GROUP BY         [x]
        1.20. HAVING
        1.21. IN               [x]
        1.22. INSERT INTO      [x]
        1.23. INNER JOIN
        1.24. LEFT JOIN
        1.25. RIGHT JOIN
        1.26. FULL JOIN
        1.27. LIKE
        1.28. ORDER BY
        1.29. SELECT           [x]
        1.30. SELECT *         [x]
        1.31. SELECT DISTINCT
        1.32. SELECT INTO
        1.33. SELECT TOP
        1.34. TRUNCATE TABLE
        1.35. UNION
        1.36. UNION ALL
        1.37. UPDATE           [x]
        1.38. WHERE            [x]
        1.39. PRAGMA           [x]


In [2]:
# Import libraries
import pandas as pd
from pandas import DataFrame
import numpy as np
import random
import datetime
from datetime import timedelta
import names                                     # pip install names
from faker.providers.person.en import Provider   # pip install faker
import sqlite3

In [3]:
conn = sqlite3.connect('./fake_db.db')
c = conn.cursor()

In [4]:
# The most basic thing is to select and view one entire table
c.execute("SELECT * FROM Hospital_features;").fetchall()

OperationalError: no such table: Hospital_features

In [ ]:
# Now, select one column from a particular table
c.execute("SELECT Patient_admitted_id FROM covid_19_death;").fetchall()

In [ ]:
# Can I select more than one column?
c.execute("SELECT Patient_admitted_id,Hospital_ID FROM covid_19_death;").fetchall()

In [ ]:
# Two columns with conditions
c.execute("SELECT NHS_Number, Birthdate FROM patient_data WHERE Postcode = 'SO15 5FL' OR Postcode = 'BE01 5SA';").fetchall()


In [ ]:
# Could I insert a new patient in the patient_data table??
# Let's first count how many patient I have before and after insert a new patient

c.execute("SELECT COUNT(ID) from patient_data ;").fetchall()

In [ ]:
# I want to be one of the patients of this database
c.execute("INSERT INTO  patient_data VALUES ('00001','000001', 'Manuel Dominguez','M','1990-04-07','Spanish', 'SO15 10FL');").fetchall()

In [ ]:
# Count again, now the number must be 2001
c.execute("SELECT COUNT(ID) from patient_data ;").fetchall()

What if we make a mistake or notice an error in a record? We can use the <code>UPDATE</code> command to change existing data in a table.

In [ ]:
c.execute("UPDATE patient_data SET 'ID'= '88888' WHERE Full_Name = 'Manuel Dominguez';").fetchall()

In [ ]:
# Let's see if the new value was introduced
c.execute("select * FROM patient_data WHERE Full_name = 'Manuel Dominguez';").fetchall()

In [ ]:
# Another useful thing to be able to do is to delete records from a table.
c.execute("DELETE FROM patient_data WHERE Full_name = 'Manuel Dominguez';").fetchall()


In [ ]:
c.execute("select * FROM patient_data WHERE Full_name = 'Manuel Dominguez';").fetchall()

No show here but I have seen how problematic if 2 or more patients were to have the same name, as these changes would apply to all people who shared the same name. I am wondering how this is solve in real patients database, by ID as a primaru key???

After some reading I have fouond that the way to overcome this in relational databases is to give each record a unique id, called a <code>primary key</code>. 

In [5]:
c.execute("PRAGMA table_info(patient_data);").fetchall()
# cid 	name 	type 	notnull 	dflt_value 	pk

[(0, 'ID', '', 1, None, 1),
 (1, 'NHS_Number', '', 0, None, 0),
 (2, 'Full_Name', '', 0, None, 0),
 (3, 'Gender', '', 0, None, 0),
 (4, 'Birthdate', '', 0, None, 0),
 (5, 'Ethnicity', '', 0, None, 0),
 (6, 'Postcode', '', 0, None, 0)]

This shows us the field name, data type, if the field can be NULL or not . We can also see any default values that are set and if a field is a primary key (pk) or not. I

To create a PRIMARY KEY constraint on the "ID" column when patient_data table already exists, I can use the following SQLite syntax.

In [6]:
c.execute("ALTER TABLE patient_data ADD PRIMARY KEY (ID);").fetchall()

OperationalError: near "PRIMARY": syntax error

I have just fouond that If your table have data you wish to add a primary key later, you cannot use the ALTER TABLE statement to create a primary key. That is not possible with SQLite. Instead, you must create a new table with the primary key and copy the data into this new table. 

In [7]:
c.execute("SELECT Full_Name FROM patient_data GROUP BY Ethnicity;").fetchall()

[('Jayla Watsica',),
 ('Osborn Senger',),
 ('Brandin Terry',),
 ('Darwin Corwin',),
 ('Amari Larson',)]

In [8]:
c.execute("SELECT Full_Name FROM patient_data GROUP BY Ethnicity;").fetchall()

[('Jayla Watsica',),
 ('Osborn Senger',),
 ('Brandin Terry',),
 ('Darwin Corwin',),
 ('Amari Larson',)]

In [11]:
# Let's get the name and NHS number of the patients that got COVID-19

c.execute("SELECT Full_Name, NHS_Number FROM patient_data INNER JOIN covid_19_admission ON covid_19_admission.Patient_admitted_id = patient_data.ID ;").fetchall()



[('Lenord Waelchi', 338171393),
 ('Ryne Mitchell', 786403777),
 ('Mack Friesen', 228011564),
 ('Maryjane Vandervort', 474502595),
 ('Harriette Daugherty', 801883343),
 ('Jacqueline Heathcote', 371685650),
 ('Anona Zboncak', 552424208),
 ('Rachel Kunze', 598994299),
 ('Elzy Bosco', 502275903),
 ('Lennie Collins', 416810429),
 ('Waino Ernser', 514736804),
 ('Deborrah Howe', 706036359),
 ('Cristin Miller', 573637105),
 ('Giuliana Rowe', 964595556),
 ('Winfield Ward', 793326617),
 ('Herbert Dicki', 808592034),
 ('Attilio Konopelski', 924009744),
 ('Alesha Howell', 620755640),
 ('Felton Upton', 446171702),
 ('Camryn Ernser', 190660292),
 ('Barb Spencer', 961306947),
 ('Renita Fay', 715836970),
 ('Zigmund Von', 619408217),
 ('Dorris Durgan', 119141523),
 ('Alesia Carroll', 572766997),
 ('Christiana Feil', 692186826),
 ('Kadyn Bartell', 716447149),
 ('Joshuah Jaskolski', 677755665),
 ('Nola Deckow', 125869915),
 ('Jalon Runte', 664323115),
 ('Romaine Welch', 591452298),
 ('Jann Schroeder', 50

### Notebook details
<br>
<i>Notebook created by <strong>Manuel Dominguez</strong> 

Creation date: May 2021<br>


In [10]:
c.execute("SELECT * FROM covid_19_admission;").fetchall()

[]